In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np
import pandas as pd
from itertools import product
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.metrics import (
    calinski_harabasz_score,
    davies_bouldin_score,
    silhouette_score,
)
from itertools import product
import pandas as pd
from tqdm import tqdm

from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import (
    calinski_harabasz_score,
    davies_bouldin_score,
    silhouette_score,
)

In [2]:

# -----------------------------
# 1. Load your data
# -----------------------------
data_path_32 = "../data/preprocessed/preprocessed_reduced_unsupervised_32.csv"
X_data_32 = pd.read_csv(data_path_32)



data_path_15 = "../data/preprocessed/preprocessed_reduced_unsupervised_15.csv"
X_data_15 = pd.read_csv(data_path_15)



print("Data shape after sampling (k-mean):", X_data_32.shape)
print("Data shape after sampling (dbscan):", X_data_15.shape)

Data shape after sampling (k-mean): (735483, 32)
Data shape after sampling (dbscan): (147097, 15)


In [ ]:



# =========================================================
# Grid search for K-Means / MiniBatch K-Means
# =========================================================
def run_kmeans_grid(
    X,
    k_values,
    output_path:str,
    init_methods=("k-means++",),
    metrics=("ch", "dbi", "wcss"),
    algorithm="kmeans",  # "kmeans" | "minibatch_kmeans"
    max_iter=300,
    n_init=10,
    batch_size=1024,     # used only for minibatch_kmeans
    random_state=42,
):
    """
    Parameters
    ----------
    X : ndarray of shape (N, d)
        Scaled input data
    k_values : list[int]
        Number of clusters to test
    init_methods : tuple[str]
        Initialization methods
    metrics : tuple[str]
        Metrics to compute: {"ch", "dbi", "silhouette", "wcss"}
    algorithm : str
        "kmeans" or "minibatch"
    max_iter : int
    n_init : int
    batch_size : int
        MiniBatchKMeans only
    random_state : int

    Returns
    -------
    results_df : pd.DataFrame
        Grid search results
    """

    if algorithm not in {"kmeans", "minibatch_kmeans"}:
        raise ValueError("algorithm must be 'kmeans' or 'minibatch_kmeans'")

    N = X.shape[0]
    results = []

    grid = list(product(k_values, init_methods))

    for K, init in tqdm(grid, desc=f"{algorithm} Grid Search"):

        if algorithm == "kmeans":
            model = KMeans(
                n_clusters=K,
                init=init,
                max_iter=max_iter,
                n_init=n_init,
                random_state=random_state,
                algorithm="lloyd",
            )
        else:  # MiniBatchKMeans
            model = MiniBatchKMeans(
                n_clusters=K,
                init=init,
                max_iter=max_iter,
                batch_size=batch_size,
                n_init=n_init,
                random_state=random_state,
            )

        labels = model.fit_predict(X)

        row = {
            "K": K,
            "init": init,
            "algorithm": algorithm,
        }

        # -----------------------
        # Selected metrics
        # -----------------------
        if "ch" in metrics:
            row["CH"] = calinski_harabasz_score(X, labels)

        if "dbi" in metrics:
            row["DBI"] = davies_bouldin_score(X, labels)

        if "silhouette" in metrics:
            # WARNING: O(N^2)
            row["Silhouette"] = silhouette_score(X, labels)

        if "wcss" in metrics:
            row["WCSS_per_point"] = model.inertia_ / N

        results.append(row)

    results_df = pd.DataFrame(results)
    results_df.to_csv(output_path, index=False)

    return results_df


In [21]:
results = run_kmeans_grid(
    X=X_data_32,
    k_values=list(range(3, 51)),  # 3,5,7,...,49
    output_path="./results/knn_metrics.csv",
    algorithm="minibatch_kmeans",  
    batch_size=4096,
    metrics=("ch", "dbi","wcss"),
)



minibatch_kmeans Grid Search: 100%|██████████| 48/48 [02:12<00:00,  2.76s/it]


In [22]:
def dominates(a, b, metrics, directions):
    """
    Returns True if solution a dominates solution b
    """
    better_or_equal = True
    strictly_better = False

    for m in metrics:
        if directions[m] == "max":
            if a[m] < b[m]:
                better_or_equal = False
                break
            elif a[m] > b[m]:
                strictly_better = True

        else:  # minimize
            if a[m] > b[m]:
                better_or_equal = False
                break
            elif a[m] < b[m]:
                strictly_better = True

    return better_or_equal and strictly_better




METRIC_DIRECTIONS = {
    "CH": "max",
    "DBI": "min",
    "Silhouette": "max",
    "WCSS_per_point": "min",
}

def extract_pareto_front(
    csv_path,
    metrics,
    directions=METRIC_DIRECTIONS,
):
    """
    Parameters
    ----------
    csv_path : str
        Path to CSV file containing grid search results
    metrics : list[str]
        Metrics to consider for Pareto dominance
    directions : dict
        Metric optimization directions ("min" or "max")

    Returns
    -------
    pareto_df : pd.DataFrame
        Non-dominated solutions
    """

    df = pd.read_csv(csv_path)

    # --- Safety checks ---
    for m in metrics:
        if m not in df.columns:
            raise ValueError(f"Metric '{m}' not found in CSV")
        if m not in directions:
            raise ValueError(f"No direction specified for metric '{m}'")

    pareto_mask = [True] * len(df)

    for i in range(len(df)):
        if not pareto_mask[i]:
            continue

        for j in range(len(df)):
            if i == j or not pareto_mask[j]:
                continue

            if dominates(df.iloc[j], df.iloc[i], metrics, directions):
                pareto_mask[i] = False
                break

    pareto_df = df[pareto_mask].reset_index(drop=True)
    return pareto_df



pareto = extract_pareto_front(
    csv_path="./results/knn_metrics.csv",
    metrics=["CH", "DBI"],
)

print(pareto)


    K       init         algorithm             CH       DBI  WCSS_per_point
0   3  k-means++  minibatch_kmeans  254988.834430  1.721380       16.684688
1   5  k-means++  minibatch_kmeans  180414.047959  1.701579       14.263527
2   8  k-means++  minibatch_kmeans  147410.138872  1.512309       11.756861
3  11  k-means++  minibatch_kmeans  126335.821244  1.505054       10.413611
4  12  k-means++  minibatch_kmeans  125704.384579  1.368123        9.811640
5  13  k-means++  minibatch_kmeans  124980.671951  1.341546        9.296829
6  14  k-means++  minibatch_kmeans  120858.929637  1.329219        9.021307
7  16  k-means++  minibatch_kmeans  115956.241494  1.277816        8.403590
8  21  k-means++  minibatch_kmeans  117611.923318  1.288624        6.730543
9  26  k-means++  minibatch_kmeans  110334.797442  1.211595        5.948190


In [ ]:
results = run_kmeans_grid(
    X=X_data_32,
    k_values=[3,5,8,11,12,14,16,21,26],  # 3,5,7,...,49
    output_path="./results/knn_silhouette.csv",
    algorithm="minibatch_kmeans",  
    batch_size=4096,
    metrics=("silhouette"),
)



minibatch_kmeans Grid Search:   0%|          | 0/9 [00:00<?, ?it/s]